
## 0) Постановка задачи

Предсказать отказ компонента пневмосистемы (APS) по телеметрии датчиков грузовиков. Это бинарная классификация: pos — целевой отказ APS; neg — другие поломки. В официальной постановке есть стоимость ошибок: FP=10, FN=500 — пропуск «плохого» грузовика сильно дороже ложной тревоги.


## 1) Чтение данных

In [ ]:

import pandas as pd
import numpy as np

train_path = "/mnt/data/aps_failure_training_set.csv"
test_path  = "/mnt/data/aps_failure_test_set.csv"

def read_aps_csv(path):
    """Читает CSV APS, пропуская преамбулу лицензии до строки 'class,....'"""
    header_line_idx = None
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for idx, line in enumerate(f):
            if line.lower().startswith("class,"):
                header_line_idx = idx
                break
    if header_line_idx is None:
        raise RuntimeError("Не найден заголовок 'class,...' в файле " + path)
    df = pd.read_csv(path, na_values=["na"], skiprows=header_line_idx, header=0)
    return df

df = read_aps_csv(train_path)
df_test_raw = read_aps_csv(test_path)

print("TRAIN shape:", df.shape)
print("TEST shape:", df_test_raw.shape)
print("Columns sample:", df.columns[:10].tolist())

y = (df["class"] == "pos").astype(int).rename("TARGET")
X = df.drop(columns=["class"])

N, d = X.shape
print(f"N={N}, d={d}, positive_rate={y.mean():.4f}")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/aps_failure_training_set.csv'

## 2) Разбиение на обучающую/тестовую

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("Train:", X_train.shape, "Test:", X_test.shape)


## 3) EDA: визуализация и основные характеристики

In [ ]:

import matplotlib.pyplot as plt

# 3.1 Class distribution
counts = y.value_counts().sort_index()
plt.figure()
plt.bar(['neg(0)', 'pos(1)'], [counts.get(0,0), counts.get(1,0)])
plt.title("Распределение классов (весь TRAIN)")
plt.xlabel("Класс")
plt.ylabel("Число объектов")
plt.show()

# 3.2 Missing by feature
missing_rate = X.isna().mean().sort_values(ascending=False)
print("Топ-20 признаков по доле пропусков:")
print(missing_rate.head(20))

plt.figure()
plt.plot(np.arange(len(missing_rate)), missing_rate.values)
plt.title("Доля пропусков по признакам (отсортировано)")
plt.xlabel("Признаки (индекс после сортировки)")
plt.ylabel("Доля пропусков")
plt.show()

# 3.3 Descriptives (head)
desc = X.describe().T
display(desc.head(15))

# 3.4 Top-15 pairwise correlations
corr = X.corr(method="pearson", min_periods=1)
vals = []
cols = corr.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        v = corr.iloc[i, j]
        if not np.isnan(v):
            vals.append((cols[i], cols[j], abs(v), v))
top_corr = sorted(vals, key=lambda x: x[2], reverse=True)[:15]
print("Топ-15 пар по |corr|:")
for a,b,absv,v in top_corr:
    print(f"{a:>8s} ~ {b:<8s} |corr|={absv:.3f}, corr={v:.3f}")

# 3.5 Rough outlier rate via IQR*3 on any feature
def row_outlier_mask(df_num):
    mask_any = np.zeros(len(df_num), dtype=bool)
    for col in df_num.columns:
        s = df_num[col]
        q1 = s.quantile(0.25)
        q3 = s.quantile(0.75)
        iqr = q3 - q1
        if pd.isna(iqr) or iqr == 0:
            continue
        low = q1 - 3 * iqr
        high = q3 + 3 * iqr
        m = (s < low) | (s > high)
        mask_any |= m.fillna(False).to_numpy()
    return mask_any

outlier_mask = row_outlier_mask(X)
print(f"Оценка доли строк-выбросов (IQR*3, ≥1 фиче): {outlier_mask.mean()*100:.2f}%")


## 4–6) Предобработка: пропуски → median, категориальных в X нет, нормализация StandardScaler

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

preproc = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])


## 7) Базовый классификатор и аргументация выбора


Используем **LogisticRegression (class_weight='balanced')**:

* Быстро и даёт вероятности → удобно **настраивать порог** под несимметричные стоимости ошибок.
* Устойчива и интерпретируема.


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, confusion_matrix

# Fast training subset
FAST_MODE = True
NEG_SAMPLE = 6000

def make_train_subset(X_tr, y_tr, neg_sample=NEG_SAMPLE):
    df_tr = X_tr.copy()
    df_tr["TARGET"] = y_tr.values
    pos = df_tr[df_tr["TARGET"] == 1]
    neg = df_tr[df_tr["TARGET"] == 0].sample(n=neg_sample, random_state=42)
    sub = pd.concat([pos, neg], axis=0).sample(frac=1.0, random_state=42).reset_index(drop=True)
    y_sub = sub["TARGET"].astype(int)
    X_sub = sub.drop(columns=["TARGET"])
    return X_sub, y_sub

if FAST_MODE:
    X_sub, y_sub = make_train_subset(X_train, y_train, NEG_SAMPLE)
    print("FAST_MODE subset:", X_sub.shape, "pos_rate:", y_sub.mean())
else:
    X_sub, y_sub = X_train, y_train
    print("Full training:", X_sub.shape)

clf = Pipeline(steps=[
    ("pre", preproc),
    ("model", LogisticRegression(max_iter=200, class_weight="balanced", solver="lbfgs"))
])
clf.fit(X_sub, y_sub)

proba = clf.predict_proba(X_test)[:, 1]
pred05 = (proba >= 0.5).astype(int)

def report(y_true, y_pred, y_score):
    acc = accuracy_score(y_true, y_pred)
    bacc = balanced_accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_score)
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    print(f"Accuracy={acc:.4f}, BalancedAcc={bacc:.4f}, ROC AUC={auc:.4f}")
    print("Confusion matrix (labels=[0,1]):
", cm)
    return acc, bacc, auc, cm

print("== LogisticRegression, threshold=0.5 ==")
acc, bacc, auc, cm = report(y_test, pred05, proba)


## 10) Борьба с дисбалансом: настройка порога по стоимости

Оптимизируем порог по формуле `TotalCost = 10*FP + 500*FN`. Перебираем 0.01…0.99.

In [ ]:

import numpy as np
from sklearn.metrics import confusion_matrix

COST_FP = 10
COST_FN = 500

thresholds = np.linspace(0.01, 0.99, 99)
best = None

for th in thresholds:
    pred = (proba >= th).astype(int)
    cm = confusion_matrix(y_test, pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    total = COST_FP*fp + COST_FN*fn
    if best is None or total < best["total"]:
        best = {"th": th, "cm": cm, "total": total}

print(f"Best threshold: {best['th']:.2f}, TotalCost={best['total']}")
print("Confusion matrix @best threshold:
", best["cm"])


## 9) (опц.) Другие модели и сравнение

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

results = []

# KNN (на подвыборке)
knn = Pipeline(steps=[("pre", preproc), ("model", KNeighborsClassifier(n_neighbors=11, weights="distance"))])
knn.fit(X_sub, y_sub)
proba_knn = knn.predict_proba(X_test)[:, 1]
pred_knn = (proba_knn >= 0.5).astype(int)
acc, bacc, auc, cm = report(y_test, pred_knn, proba_knn)
results.append(("KNN(k=11)", acc, bacc, auc))

# RandomForest (balanced_subsample)
rf = Pipeline(steps=[("impute", SimpleImputer(strategy="median")),
                    ("model", RandomForestClassifier(n_estimators=200, random_state=42,
                                                    class_weight="balanced_subsample", n_jobs=-1))])
rf.fit(X_sub, y_sub)
proba_rf = rf["model"].predict_proba(rf["impute"].transform(X_test))[:, 1]
pred_rf = (proba_rf >= 0.5).astype(int)
acc, bacc, auc, cm = report(y_test, pred_rf, proba_rf)
results.append(("RandomForest", acc, bacc, auc))

print("\nSummary (test):")
for name, acc, bacc, auc in results:
    print(f"{name:15s}  Acc={acc:.4f}  BAcc={bacc:.4f}  AUC={auc:.4f}")


## 12) Выводы


* Данные большие (60k×170), с множеством пропусков и сильным дисбалансом (≈1.7% `pos`).
* Базовый конвейер **median-impute → scale → LogisticRegression(balanced)** даёт высокий **ROC AUC** и адекватную сбалансированную точность.
* **Тюнинг порога** по стоимостной функции ожидаемо уменьшает общую стоимость за счёт повышения чувствительности к классу `pos`.
* Дальнейшие шаги: градиентный бустинг, тщательное отбор/инженерия признаков, кросс-валидация порога и более продвинутые техники борьбы с дисбалансом (разбавление, SMOTE, focal loss и т.п.).
